In [1]:
import time
import matplotlib
import rasterio as rio
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import matplotlib.axes as ax
from matplotlib import colors
from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr
import pickle
import os
import pathlib
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics

#Input the 4-Band Image Tiff File, 1-Band Tag Tiff File.
#Returns the following 4 arrays:
    #Image Training Array which are tagged and scaled (70%)
    #Image Testing Array which are tagged and scaled (30%)
    #Tag Training Array which are tagged (70%)
    #Tag Testing Array which are tagged (30%)
# You can comment out the print output to hide the info.
print("Loaded LoadTrainingData()")
def LoadTrainingData(ImgFile,TagFile):   
    #Loading the raster
    #Loading the data read() returns array with shape (bands, rows, columns)
    X_raster = rio.open(ImgFile)
    X_matrix = X_raster.read()
    #print("image_matrix is a", type(X_matrix), " with shape", X_matrix.shape)
    scaler = StandardScaler()
    #Image Data Flattening
    X_flat = np.array([X_matrix[0].flatten(),
                              X_matrix[1].flatten(),
                              X_matrix[2].flatten(),
                              X_matrix[3].flatten()]).T
    X_flat = scaler.fit_transform(X_flat)
    #Loading classification data
    #classification_raster = rio.open('/Users/wto/Downloads/hxip_m_3711954_sw_11_100_nov29_ag.tif')
    Y_raster = rio.open(TagFile)
    Y_matrix = Y_raster.read().astype(int)
    #Tagged Data Flattening
    Y_flat = Y_matrix[0].flatten()
    
    
    print('unburnables tagged ',len(Y_flat[Y_flat==1]))
    print('green grass tagged ',len(Y_flat[Y_flat==2]))
    print('dry grass tagged ',len(Y_flat[Y_flat==3]))
    print('greentree tagged ',len(Y_flat[Y_flat==4]))
    print('dead trees tagged ',len(Y_flat[Y_flat==5]))
  
    #Cuts (Train/Test Split):
    Y_flat_tagged = Y_flat[Y_flat!=0]

    #print("X_flat_tagged.shape", X_flat_tagged.shape)
    #print("Y_flat_tagged.shape", Y_flat_tagged.shape)

    X_train, X_test, Y_train, Y_test = train_test_split(X_flat[Y_flat!=0], Y_flat[Y_flat!=0], 
                                                        stratify = Y_flat[Y_flat!=0], 
                                                        random_state = 42,test_size= 0.3)
    
    #print("The Training Image is shaped", X_train.shape)
    #print("The Test Image is shaped", X_test.shape)
    #print("The Training Tag is shaped", Y_train.shape)
    #print("The Test Tag is shaped", Y_test.shape)

    return X_train,X_test, Y_train, Y_test, X_matrix, X_flat,Y_matrix, Y_flat
#################################################################################


## Function to run the RMSD Analysis.
#Three Inputs: 
    # The NIR Band of the image
    # The Predicted Classification Y_predict
print("Loaded RMS_of_band5x5()")
def RMSD_of_band5x5(X_matrix, Y_matrix):
    #Padding the image
    prediction_matrix = np.zeros((len(X_matrix)+4, len(X_matrix[0])+4))
    matrix = np.zeros((len(X_matrix)+4, len(X_matrix[0])+4))
    matrix[2:len(X_matrix)+2,2:len(X_matrix[0])+2]=X_matrix
    prediction_matrix[2:len(X_matrix)+2,2:len(X_matrix[0])+2]  = Y_matrix
    prediction_matrix = prediction_matrix.astype('int')
    
    #Initializing RMS Matrix
    RMS_matrix=np.zeros(X_matrix.shape)
    
    for i in range(len(X_matrix)):
        for j in range(len(X_matrix[0])):
            if Y_matrix[i,j]==4:
                image_chip = matrix[i:i+5,j:j+5].flatten()
                image_chip_pred = prediction_matrix[i:i+5,j:j+5].flatten()
                
                calc_image=image_chip[image_chip_pred==4]
                RMS=calc_image.std()
                
                RMS_matrix[i,j]=RMS
            else:
                RMS_matrix[i,j]=-1
    return RMS_matrix


#### Print the accuracies provide 2 inputs:
### 1) the prediction 1D array
### 2) the tagging 1D array
### Returns the number tagged of each class and burnable vs unburnables
### AND the number that are correct.
print("Loaded PrintAcc()")
def PrintAcc(Y_predict, Y_flat):
    n=np.zeros(6)
    ncor=np.zeros(6)

    for ipixel in range(len(Y_predict)):
        if(Y_flat[ipixel]==1):
            n[1]+=1
            if(Y_predict[ipixel]==1):
                ncor[1]+=1
        if(Y_flat[ipixel]==2):
            n[2]+=1
            if(Y_predict[ipixel]==2):
                ncor[2]+=1
        if(Y_flat[ipixel]==3):
            n[3]+=1
            if(Y_predict[ipixel]==3):
                ncor[3]+=1
        if(Y_flat[ipixel]==4):
            n[4]+=1
            if(Y_predict[ipixel]==4):
                ncor[4]+=1
        if(Y_flat[ipixel]==5):
            n[5]+=1
            if(Y_predict[ipixel]==5):
                ncor[5]+=1

    print("Unburnable:","{:.1f}%".format(ncor[1]/n[1]*100))
    print("GreenGrass:","{:.1f}%".format(ncor[2]/n[2]*100))
    print("DryGrass:","{:.1f}%".format(ncor[3]/n[3]*100))
    print("GreenTree:","{:.1f}%".format(ncor[4]/n[4]*100))
    print("DeadTree:","{:.1f}%".format(ncor[5]/n[5]*100))
    print("Burnable:","{:.1f}%".format(sum(ncor[2:5])/sum(n[2:5])*100))
    print("Overall:","{:.1f}%".format(sum(ncor[1:5])/sum(n[1:5])*100))
    
    return n, ncor

### Saving the prediction as a tiff file with the inputFile's geocoordinates.
# Input 1) the prediction flat data , 2) file with the same geocoordinates 3) Output Filename
print("Loaded SavePredictTif()")
def SavePredictTif(Y_predict, imageFile, outFile):
    print("Saving as tif raster file")
    image_raster = rio.open(imageFile)
    image_data = image_raster.read()

    #Getting Coordinate Meshgrid
    #print("Starting Copying coordinate mesh data")    
    #print(type(image_raster.xy))
    #print("image_data.shape",image_data.shape)


    Y_predict_matrix = np.reshape(Y_predict,image_data[0].shape)
    #print("Y_predict_matrix.shape",Y_predict_matrix.shape)

    cols, rows = np.meshgrid(np.arange(image_raster.width), np.arange(image_raster.height))
    coordinate_data_x, coordinate_data_y = np.asarray(rio.transform.xy(image_raster.transform, rows, cols))

    xmin,ymin,xmax,ymax = [coordinate_data_x.min()-0.5,
                           coordinate_data_y.min()+0.5,
                           coordinate_data_x.max()-0.5,
                           coordinate_data_y.max()+0.5]
    nrows = Y_predict_matrix.shape[0]
    ncols = Y_predict_matrix.shape[1]
    xres = round((xmax-xmin)/float(ncols))
    yres = round((ymax-ymin)/float(nrows))
    geotransform=(xmin,xres,0,ymax,0,-yres)   

    output_raster = gdal.GetDriverByName('GTiff').Create(outFile,
                                                         ncols, nrows, 1,
                                                         gdal.GDT_Float32)
    output_raster.SetGeoTransform(geotransform)  
    srs = osr.SpatialReference()                 
    srs.ImportFromEPSG(32611)                    

    output_raster.SetProjection( srs.ExportToWkt() )
    output_raster.GetRasterBand(1).WriteArray(Y_predict_matrix)

    output_raster.FlushCache()

Loaded LoadTrainingData()
Loaded Load2TrainingData()
Loaded RMS_of_band5x5()
Loaded PrintAcc()
Loaded SavePredictTif()
